### **Dataset from huggingface**

In [2]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPUs detected: {[gpu.name for gpu in gpus]}")
    except RuntimeError as e:
        print(f"Error setting up GPU memory growth: {e}")
else:
    print("No GPU detected. Using CPU.")


No GPU detected. Using CPU.


In [ ]:
!pip install -U datasets

### **Dataset 1 : trashnet_enhanced** (19.9k)

In [ ]:
# from datasets import load_dataset
# data1 = load_dataset("edwinpalegre/trashnet_enhanced")

### **Dataset 2 : small trashnet** ( 2.53k)

In [ ]:
# from datasets import load_dataset
# data2 = load_dataset("kuchidareo/small_trashnet")

### **Datset 3 : trashnet** (5.5k)

In [ ]:
# from datasets import load_dataset
# data3 = load_dataset("garythung/trashnet")

## **Importing Libraries**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from PIL import Image
from collections import Counter

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

In [ ]:
from tensorflow import keras
from keras.layers import Input, Activation, Add, Dense, Conv2D, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import BatchNormalization, Dropout
from keras.models import Model
from keras.utils import plot_model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping,ModelCheckpoint

from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array# type: ignore
from tensorflow.keras.applications import ResNet50,VGG16,MobileNetV2,InceptionV3 # type: ignore
from tensorflow.keras.applications.resnet50 import preprocess_input # type: ignore
from tensorflow.keras.applications.vgg16 import preprocess_input # type: ignore
from tensorflow.keras.preprocessing import image # type: ignore

## Setting up Directory to save model data

In [ ]:
os.makedirs('test_dir', exist_ok=True)

In [ ]:
project_name = 'Waste_Classification'

model_names = [
    'Custom_CNN_From_Scratch',
    'VGG16_Transfer_Learning',
    'ResNet50_Transfer_Learning'
]

base_dir = '/content/'

project_dir = os.path.join(base_dir, project_name)
os.makedirs(project_dir, exist_ok=True)

for each_model in model_names:
    model_dir = os.path.join(project_dir, each_model)
    os.makedirs(model_dir, exist_ok=True)

print(f'Project directory structure created at: {project_dir}')

## **Analysis of Data**

## **Dataset : small_trashnet**

In [ ]:
from datasets import load_dataset
dataset = load_dataset("kuchidareo/small_trashnet")

In [ ]:
label_mapping = {
    0: 'cardboard',
    1: 'glass',
    2: 'metal',
    3: 'paper',
    4: 'plastic',
    5: 'trash'
}

In [ ]:
print("--------------------------")
print("Small Trashnet Dataset")
print("--------------------------\n")

all_dimensions_set = set()

label_image_count = {}

total_images = 0

for item in dataset['train']:
    label = item['label']
    image = item['image']

    if label not in label_image_count:
        label_image_count[label] = 0
    label_image_count[label] += 1
    total_images += 1

    if hasattr(image, 'size') and hasattr(image, 'getbands'):
        width, height = image.size
        channels = len(image.getbands())
        all_dimensions_set.add((width, height, channels))
    else:
        print(f"Warning: Skipping item with non-image 'image' field type: {type(image)}")


for label, count in label_image_count.items():
    print(f"{label} folder contains {count} images.")

print(f"\nTotal images in the dataset: {total_images}")

if len(all_dimensions_set) == 1:
    if all_dimensions_set:
        width, height, channels = all_dimensions_set.pop()
        print(f"All images in the dataset have the same dimensions: {width}x{height} with {channels} color channels.")
    else:
        print("No valid images found to determine dimensions.")
else:
    print("The images in the dataset have different dimensions or color channels.")

## **Images of Each Category**

In [ ]:
label_mapping = {
    0: 'cardboard',
    1: 'glass',
    2: 'metal',
    3: 'paper',
    4: 'plastic',
    5: 'trash'
}

In [ ]:
garbage_types = sorted(set([item['label'] for item in dataset['train']]))

for garbage_type in garbage_types:
    filtered_dataset = [item for item in dataset['train'] if item['label'] == garbage_type]

    images = []
    for i, item in enumerate(filtered_dataset):
        if i >= 7:
            break
        images.append(item['image'])

    if not images:
        continue

    fig, axs = plt.subplots(1, len(images), figsize=(11, 3))

    for i, img in enumerate(images):
        axs[i].imshow(img)
        axs[i].axis('off')

    fig.suptitle(f"{label_mapping[garbage_type]}", fontsize=15, y=1.0)
    plt.tight_layout()
    plt.show()

In [ ]:
labels = [label_mapping[example['label']] for example in dataset['train']]

In [ ]:
class_counts = Counter(labels)
plt.figure(figsize=(8,5))
plt.bar(class_counts.keys(),
        class_counts.values(),
        color='skyblue',
        edgecolor='k',
        width=0.5)

plt.xlabel("Class")
plt.ylabel("Number of Images")
plt.title("Number of Images per Class")
plt.xticks(rotation=0)
plt.show()

In [ ]:
textprops = {'fontsize':11 , }
wedgeprops = {'linewidth':2 , 'width':1,'edgecolor':'k'}

plt.figure(figsize=(6,6))
plt.pie(class_counts.values(),
        labels=class_counts.keys(),
        autopct='%1.1f%%',
        colors=['yellow','lightgreen','blue','indigo','violet','red'],
        textprops=textprops,
        wedgeprops=wedgeprops
        )
plt.title("Percentage of Total Images for Each Class")
plt.show()

## **Data Splitting**

In [ ]:
dataset.column_names

In [ ]:
data = []

for item in dataset['train']:
    label_num = item['label']
    label = label_mapping[label_num]
    data.append((item['image'], label))

df = pd.DataFrame(data, columns=['image', 'label'])

In [ ]:
print(pd.DataFrame(df.head()))

In [ ]:
print(df.info())

In [ ]:
def extract_labels(example):
    return {'labels': example['label']}

dataset = dataset.map(extract_labels, remove_columns=['label'])

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
print(f"Number of images in the training set: {len(train_df)}")
print(f"Number of images in the validation set: {len(val_df)}")

In [ ]:
overall_distribution = df['label'].value_counts(normalize=True) * 100

train_distribution = train_df['label'].value_counts(normalize=True) * 100

val_distribution = val_df['label'].value_counts(normalize=True) * 100

print("Class distribution in the entire dataset:\n")
print(overall_distribution.round(2))
print('-'*40)

print("\nClass distribution in the training set:\n")
print(train_distribution.round(2))
print('-'*40)

print("\nClass distribution in the validation set:\n")
print(val_distribution.round(2))

In [ ]:
sns.set(style="whitegrid")

fig, axes = plt.subplots(1,3, figsize=(15,6))

# Plot for overall distribution
axes[0].bar(overall_distribution.index, overall_distribution.values, color='blue')
axes[0].set_title("Overall Class Distribution")
axes[0].set_xlabel("Class", fontsize=12, fontweight='bold')
axes[0].set_ylabel("Percentage", fontsize=12, fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)

# Plot for train distribution
axes[1].bar(train_distribution.index, train_distribution.values, color='green')
axes[1].set_title("Train Class Distribution")
axes[1].set_xlabel("Class",fontsize=12, fontweight='bold')
axes[1].set_ylabel("Percentage",fontsize=12, fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)

# Plot for validation distribution
axes[2].bar(val_distribution.index, val_distribution.values, color='red')
axes[2].set_title("Validation Class Distribution")
axes[2].set_xlabel("Class",fontsize=12, fontweight='bold')
axes[2].set_ylabel("Percentage",fontsize=12, fontweight='bold')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Data Augmentation & Rescaling

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,                    
    rotation_range=45,                 
    width_shift_range=0.15,             
    height_shift_range=0.15,            
    zoom_range=0.15,                   
    horizontal_flip=True,              
    shear_range=0.05,                   
    brightness_range=[0.9, 1.1],        
    channel_shift_range=10,            
    fill_mode='nearest'                
)

val_datagen = ImageDataGenerator(rescale=1./255)

## Generating Batches of Image

In [ ]:
import os

output_dir = "image"
os.makedirs(output_dir, exist_ok=True)

def save_image_and_get_path(row):
    img_path = os.path.join(output_dir, f"{row.name}.jpg")
    if isinstance(row["image"], str):
        return row["image"]
    elif hasattr(row["image"], "save"):
        row["image"].save(img_path)
        return img_path
    else:
        return None

train_df["filepath"] = train_df.apply(save_image_and_get_path, axis=1)
val_df["filepath"] = val_df.apply(save_image_and_get_path, axis=1)

train_df = train_df.dropna(subset=["filepath"])
val_df = val_df.dropna(subset=["filepath"])

train_df["filepath"] = train_df["filepath"].astype(str)
val_df["filepath"] = val_df["filepath"].astype(str)

x_col="filepath"

print(train_df["filepath"].head())
print(type(train_df["filepath"].iloc[0]))

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,                 
    x_col="filepath",                    
    y_col="label",                       
    target_size=(384, 384),             
    batch_size=32,                       
    class_mode='categorical',            
    seed=42,                             
    shuffle=False                        
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,                    
    x_col="filepath",                   
    y_col="label",                     
    target_size=(384, 384),              
    batch_size=32,                      
    class_mode='categorical',           
    seed=42,                             
    shuffle=False                        

In [ ]:
print(f"Number of batches in train_generator: {len(train_generator)}")
print(f"Number of batches in val_generator: {len(val_generator)}")

In [ ]:
num_batches = len(train_generator)

total_augmented_images = num_batches * train_generator.batch_size

print(f"Total augmented images per epoch: {total_augmented_images}")

In [ ]:
img_width, img_height = 48, 48
num_classes = 6

In [ ]:
image_path = 'image/2516.jpg'

img = load_img(image_path, color_mode='grayscale', target_size=(img_width, img_height)) 
img_array = img_to_array(img) 
img_array = img_array.reshape((1,) + img_array.shape)  

fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(10, 4))
axes[0].imshow(img_array[0, :, :, 0], cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

for i, ax in enumerate(axes.flat[1:]): 
    aug_iter = train_datagen.flow(img_array, batch_size=1)
    aug_img = next(aug_iter)[0] 
    ax.imshow(aug_img[:, :, 0], cmap='gray')
    ax.set_title(f'Augmented Image {i+1}')
    ax.axis('off')

plt.tight_layout()
plt.show()

## Addressing Dataset Imbalances

In [ ]:
class_labels = train_df['label'].unique()
class_labels

In [ ]:
weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=train_df['label'])
weights

In [ ]:
class_weights = dict(zip(train_generator.class_indices.values(), weights))
class_weights

## Model From Scratch

In [ ]:
def residual_block(X, kernel_size, filters, reduce=False, stride=2):

    F1, F2, F3 = filters

    X_shortcut = X

    if reduce:
        X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (stride,stride), padding = 'valid', kernel_initializer='he_normal')(X)
        X = BatchNormalization(axis = 3)(X)
        X = Activation('relu')(X)

        X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (stride,stride), padding = 'valid', kernel_initializer='he_normal')(X_shortcut)
        X_shortcut = BatchNormalization(axis = 3)(X_shortcut)
    else:
        X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer='he_normal')(X)
        X = BatchNormalization(axis = 3)(X)
        X = Activation('relu')(X)

    X = Conv2D(filters = F2, kernel_size = (kernel_size, kernel_size), strides = (1,1), padding = 'same', kernel_initializer='he_normal')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer='he_normal')(X)
    X = BatchNormalization(axis = 3)(X)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X


In [ ]:
def model_scratch(input_shape, classes):

    X_input = Input(input_shape)

    # Block 1
    X = Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer='he_normal')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Block 2
    X = residual_block(X, 3, [64, 64, 256], reduce=True, stride=1)
    X = residual_block(X, 3, [64, 64, 256])
    X = residual_block(X, 3, [64, 64, 256])

    # Block 3
    X = residual_block(X, 3, [128, 128, 512], reduce=True, stride=2)
    X = residual_block(X, 3, [128, 128, 512])
    X = residual_block(X, 3, [128, 128, 512])
    X = residual_block(X, 3, [128, 128, 512])

    # Block 4
    X = residual_block(X, 3, [256, 256, 1024], reduce=True, stride=2)
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])

    # Block 5
    X = residual_block(X, 3, [512, 512, 2048], reduce=True, stride=2)
    X = residual_block(X, 3, [512, 512, 2048])
    X = residual_block(X, 3, [512, 512, 2048])

    X = GlobalAveragePooling2D()(X)

    X = Dense(classes, activation='softmax')(X)

    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
def Modified_model_scratch(input_shape, classes):

    X_input = Input(input_shape)

    X = Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer='he_normal')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = residual_block(X, 3, [64, 64, 256], reduce=True, stride=1)
    X = residual_block(X, 3, [64, 64, 256])
    X = residual_block(X, 3, [64, 64, 256])

    X = residual_block(X, 3, [128, 128, 512], reduce=True, stride=2)
    X = residual_block(X, 3, [128, 128, 512])
    X = residual_block(X, 3, [128, 128, 512])
    X = residual_block(X, 3, [128, 128, 512])

    X = residual_block(X, 3, [256, 256, 1024], reduce=True, stride=2)
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])


    X = residual_block(X, 3, [512, 512, 2048], reduce=True, stride=2)
    X = residual_block(X, 3, [512, 512, 2048])
    X = residual_block(X, 3, [512, 512, 2048])

    X = GlobalAveragePooling2D()(X)

    X = Dropout(0.5)(X)

    X = Dense(classes, activation='softmax')(X)

    model = Model(inputs = X_input, outputs = X, name='Modified_Model')

    return model

In [ ]:
input_shape = (384, 384, 3)
num_classes = 6

modified_model = Modified_model_scratch(input_shape=input_shape, classes=num_classes)

In [ ]:
modified_model.summary()

In [ ]:
modified_model.compile(optimizer='adam', #0.001
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

In [ ]:
cnn_path = '/content/Waste_Classification/Custom_CNN_From_Scratch'
name = 'ModelFromScratch.keras'
chk_path = os.path.join(cnn_path, name)
chk_path

In [ ]:
checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             verbose=1,
                             mode='min',
                             monitor='val_loss')

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.5,
                              patience=15,
                              min_lr=0.00001)

early_stopping = EarlyStopping(monitor='val_loss',
                               mode='min',
                               patience=50,
                               restore_best_weights=True,
                               verbose=1)

In [ ]:
print("Assigned Class Weights:")
class_weights

In [ ]:
num_epochs = 50

history = modified_model.fit(train_generator,
                            steps_per_epoch=len(train_generator),
                            epochs=num_epochs,
                            validation_data=val_generator,
                            validation_steps=len(val_generator),
                            class_weight=class_weights,
                            callbacks=[checkpoint,reduce_lr, early_stopping],
                            verbose=1)

In [ ]:
def plot_learning_curves(history, start_epoch=5):
    df = pd.DataFrame(history.history)
    df = df.iloc[start_epoch-1:]
    sns.set(rc={'axes.facecolor': '#f0f0fc'}, style='darkgrid')

    plt.figure(figsize=(15,6))
    plt.subplot(1, 2, 1)
    sns.lineplot(x=df.index, y=df['loss'], color='green', label='Train Loss')
    sns.lineplot(x=df.index, y=df['val_loss'], color='red', linestyle='--', label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Evolution')

    plt.subplot(1, 2, 2)
    sns.lineplot(x=df.index, y=df['accuracy'], color='darkblue', label='Train Accuracy')
    sns.lineplot(x=df.index, y=df['val_accuracy'], color='red', linestyle='--', label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Evolution')

    plt.show()

### plot curves

In [ ]:
plot_learning_curves(history)

### Evaluate model performance

In [ ]:
def evaluate_model_performance(model, val_generator, class_labels):

    true_labels = val_generator.classes

    class_labels = list(val_generator.class_indices.keys())

    predictions = model.predict(val_generator, steps=len(val_generator))

    predicted_labels = np.argmax(predictions, axis=1)

    true_labels = val_generator.classes

    report = classification_report(true_labels, predicted_labels, target_names=class_labels)
    print(report)
    print('\n')

    colors = ["white", "royalblue"]
    cmap_cm = LinearSegmentedColormap.from_list("cmap_cm", colors)

    cm = confusion_matrix(true_labels, predicted_labels)

    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, cmap=cmap_cm, fmt='d', xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
evaluate_model_performance(modified_model, val_generator, class_labels)

## Testing Model

In [ ]:
model = keras.models.load_model("Models/Custom_CNN_From_Scratch/50 epochs/ModelFromScratch.keras")

In [ ]:
test_loss, test_accuracy = modified_model.evaluate(val_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

## ## ROC-AUC Curve

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

y_true = val_generator.classes
y_pred_probs = modified_model.predict(val_generator)

y_true_bin = label_binarize(y_true, classes=np.unique(y_true))
n_classes = y_true_bin.shape[1]

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(10, 8))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'{label_mapping[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve for Each Class')
plt.legend(loc="lower right")
plt.show()

# VGG16 with Transfer Learning

### Loading VGG16 with pretrained weights

In [ ]:
base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(384, 384, 3))
base_model_vgg.summary()

In [ ]:
for layer in base_model_vgg.layers:
    layer.trainable = False

In [ ]:
x = base_model_vgg.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(6, activation='softmax')(x)

In [ ]:
transfer_vgg16_model = Model(inputs=base_model_vgg.input, 
                             outputs=x)

transfer_vgg16_model.compile(optimizer=Adam(learning_rate=0.0001),
                             loss='categorical_crossentropy', 
                             metrics=['accuracy'])

transfer_vgg16_model.summary()

### Applying VGG16 for specific Image Preprocessing

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=60,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.20,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.05,
    brightness_range=[0.9, 1.1],
    channel_shift_range=10,
    fill_mode='nearest',
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
)

In [ ]:
val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="label",
    target_size=(224, 224),
    batch_size=8,  
    class_mode='categorical',
    seed=42,
    shuffle=False
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="filepath",
    y_col="label",
    target_size=(224, 224),
    batch_size=8,  
    class_mode='categorical',
    seed=42,
    shuffle=False
)


In [ ]:
cnn_path = 'Models/VGG16_Transfer_Learning'
name = 'VGG16TransferLearning.keras'
chk_path = os.path.join(cnn_path, name)


### Fine-tuning the Transfer Learning VGG16 Model

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.5, 
                              patience=5, 
                              min_lr=1e-5)

early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=15, 
                               restore_best_weights=True,
                                verbose=1)
checkpoint = ModelCheckpoint(filepath=chk_path, 
                             save_best_only=True, 
                             verbose=1, 
                             monitor='val_loss', 
                             mode='min')

In [ ]:
num_epochs = 35
history = transfer_vgg16_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    class_weight=class_weights,
    callbacks=[reduce_lr, early_stopping, checkpoint]
)

In [ ]:
test_loss, test_accuracy = transfer_vgg16_model.evaluate(val_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

### Visualizing the Learning Curves

In [ ]:
plot_learning_curves(history, start_epoch=2)

### Performance Metrics Assessment

In [ ]:
evaluate_model_performance(transfer_vgg16_model, val_generator, class_labels)

## ROC-AUC Curve

In [ ]:
y_true = val_generator.classes
y_pred_probs = transfer_vgg16_model.predict(val_generator)

y_true_bin = label_binarize(y_true, classes=np.unique(y_true))
n_classes = y_true_bin.shape[1]

fpr, tpr, roc_auc = dict(), dict(), dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(10, 8))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'{label_mapping[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve for Each Class')
plt.legend(loc="lower right")
plt.show()

# ResNet50 with Transfer Learning

## Loading ResNet50 with pretrained weights

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(384, 384, 3))

In [ ]:
base_model.summary()

In [ ]:
len(base_model.layers)

In [ ]:
for i, layer in enumerate(base_model.layers):
    if 140 <= i <= 150:
        print(i, layer.name)

In [ ]:
for layer in base_model.layers[:143]:
    layer.trainable = False

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(6, activation='softmax')(x)

transfer_resnet50_model = Model(inputs=base_model.input, outputs=x)

transfer_resnet50_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
plot_model(transfer_resnet50_model, show_shapes=False, show_layer_names=False, dpi=72)

In [ ]:
transfer_resnet50_model.summary()

## Applying ResNet50-specific Image Preprocessing

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=60,                 
    width_shift_range=0.15,            
    height_shift_range=0.15,           
    zoom_range=0.20,                 
    horizontal_flip=True,             
    vertical_flip=True,                 
    shear_range=0.05,                   
    brightness_range=[0.9, 1.1],        
    channel_shift_range=10,            
    fill_mode='nearest',                 
    preprocessing_function=preprocess_input  
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,                 
    x_col="filepath",                   
    y_col="label",                     
    target_size=(384, 384),            
    batch_size=32,                     
    class_mode='categorical',           
    seed=42,                            
    shuffle=False                      
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,                   
    x_col="filepath",                  
    y_col="label",                      
    target_size=(384, 384),              
    batch_size=32,                      
    class_mode='categorical',          
    seed=42,                            
    shuffle=False                      
)


In [ ]:
cnn_path = 'Waste_Classification/ResNet50_Transfer_Learning'
name = 'ResNet50TransferLearning.keras'
chk_path = os.path.join(cnn_path, name)
chk_path

## Fine-tuning the Transfer Learning ResNet50 Model

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.5, patience=5, 
                              min_lr=0.00001)

early_stopping = EarlyStopping(monitor='val_loss', 
                               mode='min', patience=15, 
                               restore_best_weights=True,
                               verbose=1)

checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             verbose=1,
                             mode='min',
                             monitor='val_loss')

In [ ]:
num_epochs = 30

history = transfer_resnet50_model.fit(train_generator,
                                      steps_per_epoch=len(train_generator),
                                      epochs=num_epochs,
                                      validation_data=val_generator,
                                      validation_steps=len(val_generator),
                                      class_weight=class_weights,
                                      callbacks=[reduce_lr, early_stopping,checkpoint])


In [ ]:
test_loss, test_accuracy = transfer_resnet50_model.evaluate(val_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

## Visualizing the Learning Curves

In [ ]:
plot_learning_curves(history, start_epoch=2)

## Performance Metrics Assessment

In [ ]:
evaluate_model_performance(transfer_resnet50_model, val_generator, class_labels)

## ROC-AUC Curve

In [ ]:
y_true = val_generator.classes
y_pred_probs = transfer_resnet50_model.predict(val_generator)

In [ ]:
y_true_bin = label_binarize(y_true, classes=np.unique(y_true))
n_classes = y_true_bin.shape[1]

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(10, 8))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'{label_mapping[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve for Each Class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Next possible models for this project
#  VGG19
# ResNet101
# MobileNetV2
# Xception
# InceptionV3
# DenseNet169

# MobileNetV2 with Transfer Learning

## Loading MobileNetV2 with pretrained weights

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.summary()


In [ ]:
# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(6, activation='softmax')(x)

In [ ]:
transfer_mobilenetv2_model = Model(inputs=base_model.input, outputs=x)
transfer_mobilenetv2_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
plot_model(transfer_mobilenetv2_model, show_shapes=False, show_layer_names=False, dpi=72)
transfer_mobilenetv2_model.summary()


## Applying MobileNetV2-specific Image Preprocessing

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=60,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.20,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.05,
    brightness_range=[0.9, 1.1],
    channel_shift_range=10,
    fill_mode='nearest',
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    seed=42,
    shuffle=False
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="filepath",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    seed=42,
    shuffle=False
)

In [ ]:
# Checkpoint path
cnn_path = 'Waste_Classification/MobileNetV2_Transfer_Learning'
name = 'MobileNetV2TransferLearning.keras'
chk_path = os.path.join(cnn_path, name)
os.makedirs(cnn_path, exist_ok=True)


## Fine-tuning the Transfer Learning MobileNetV2 Model

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
checkpoint = ModelCheckpoint(filepath=chk_path, save_best_only=True, verbose=1, monitor='val_loss', mode='min')

In [ ]:
num_epochs = 50
history = transfer_mobilenetv2_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    class_weight=class_weights,
    callbacks=[reduce_lr, early_stopping, checkpoint]
)


In [ ]:
test_loss, test_accuracy = transfer_mobilenetv2_model.evaluate(val_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

## Visualizing the Learning Curves

In [ ]:
plot_learning_curves(history, start_epoch=2)

## Performance Metrics Assessment

In [ ]:
evaluate_model_performance(transfer_mobilenetv2_model, val_generator, class_labels)

## ROC-AUC Curve

In [ ]:
y_true = val_generator.classes
y_pred_probs = transfer_mobilenetv2_model.predict(val_generator)

y_true_bin = label_binarize(y_true, classes=np.unique(y_true))
n_classes = y_true_bin.shape[1]

fpr, tpr, roc_auc = dict(), dict(), dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(10, 8))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'{label_mapping[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve for Each Class')
plt.legend(loc="lower right")
plt.show()

## Inception V3 with Transfer Learning

## Loading Inception V3 with pretrained weights

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
base_model.summary()

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(6, activation='softmax')(x)

In [ ]:
transfer_inceptionv3_model = Model(inputs=base_model.input, outputs=x)
transfer_inceptionv3_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
plot_model(transfer_inceptionv3_model, show_shapes=False, show_layer_names=False, dpi=72)
transfer_inceptionv3_model.summary()

## Applying Inception-V3 specific Image Preprocessing

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=60,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.20,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.05,
    brightness_range=[0.9, 1.1],
    channel_shift_range=10,
    fill_mode='nearest',
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input
)

val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="label",
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    seed=42,
    shuffle=False
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="filepath",
    y_col="label",
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    seed=42,
    shuffle=False
)

In [ ]:
cnn_path = 'Waste_Classification/InceptionV3_Transfer_Learning'
name = 'InceptionV3TransferLearning.keras'
chk_path = os.path.join(cnn_path, name)
os.makedirs(cnn_path, exist_ok=True)

## Fine-tuning the Transfer Learning Inception-V3 Model

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
checkpoint = ModelCheckpoint(filepath=chk_path, save_best_only=True, verbose=1, monitor='val_loss', mode='min')

In [ ]:
num_epochs = 50
history = transfer_inceptionv3_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    class_weight=class_weights,
    callbacks=[reduce_lr, early_stopping, checkpoint]
)

In [ ]:
test_loss, test_accuracy = transfer_inceptionv3_model.evaluate(val_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

## Visualizing the Learning Curves

In [ ]:
plot_learning_curves(history, start_epoch=2)

## Performance Metrics Assessment

In [ ]:
evaluate_model_performance(transfer_inceptionv3_model, val_generator, class_labels)

In [ ]:
y_true = val_generator.classes
y_pred_probs = transfer_inceptionv3_model.predict(val_generator)

y_true_bin = label_binarize(y_true, classes=np.unique(y_true))
n_classes = y_true_bin.shape[1]

fpr, tpr, roc_auc = dict(), dict(), dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(10, 8))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'{label_mapping[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve for Each Class')
plt.legend(loc="lower right")
plt.show()
